```
python -m drivingppo.simsim
```

In [ ]:
# 학습된 모델 확인

import time
from typing import Literal, Callable
import random
from random import randint

from drivingppo.common import set_seed
from drivingppo.world import World, Car, OBSTACLE_VALUE, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.environment import WorldEnv
from drivingppo.model import run

import numpy as np

from stable_baselines3 import PPO

LOG_DIR = "./logs/"
CHECKPOINT_DIR = './checks/'

from world_samples import gen_0, gen_1, gen_2, generate_random_world_plain


import math
pi = math.pi
pi2 = pi*2


MAP_W, MAP_H = 150, 150

W_CONFIG = {
    'map_border': False,
    'near': 2.5,
    'far': 999.9,
}

def runt(
        world_generator:Callable[[], World],
        model:PPO|str,
        time_spd=2.0,
        time_step=111,
        step_per_control=3,
        auto_close_at_end=True,
    ):

    env = WorldEnv(
        world_generator=world_generator,
        time_step=time_step,
        step_per_control=1,
        render_mode='debug',
        auto_close_at_end=auto_close_at_end
    )

    if type(model) == str:
        model = PPO.load(CHECKPOINT_DIR+model, env=env)
    assert isinstance(model, PPO)

    obs, info = env.reset()
    terminated = False
    truncated = False
    episode_reward = 0.0

    while not terminated and not truncated:

        # action, _ = model.predict(obs, deterministic=True)  # 에이전트가 행동 선택
        action = np.array([-0.0, 0.5])
        for _ in range(step_per_control):
            obs, reward, terminated, truncated, info = env.step(action)  # 행동 실행
            episode_reward += reward
            env.render()  # 시각화 호출
            time.sleep(time_step / 1000.0 / time_spd)# 시각화 프레임을 위해 딜레이 추가
            if terminated or truncated: break

    print(f"에피소드 종료. 총 보상: {episode_reward:.2f}")

    env.close()

def testew():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 30),
            (20, 30),
            (50, 50),
            (60, 40),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w

def cpetest():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 30),
            (10, 50),
            (10, 70),
            (40, 50),
            (50, 80),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w

def gen_0(): return generate_random_world_plain(map_h=150, map_w=150, num=7, wpoint_dist_min=6,   wpoint_dist_max=12, ang_init='p',    ang_lim=pi*0.5, spd_init=0)
def gen_1(): return generate_random_world_plain(map_h=300, map_w=300, num=7, wpoint_dist_min=10,  wpoint_dist_max=45, ang_init='half', ang_lim=pi*1.0, spd_init=0)
def gen_2(): return generate_random_world_plain(map_h=300, map_w=300, num=7, wpoint_dist_min=10,  wpoint_dist_max=65, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')

prefix = 'pcnn'

set_seed(1)
# run(gen_0, f'{prefix}-100--0', auto_close_at_end=False)
# run(gen_1, f'{prefix}-100--1', auto_close_at_end=False)
# run(gen_1, f'{prefix}-100--2', auto_close_at_end=False)
run(gen_1, f'dppo', auto_close_at_end=False)

print('#########')

In [ ]:
# 맵 생성 확인
import math
pi = math.pi

from drivingppo.world import create_empty_map
from drivingppo.simsim import WorldController, World, Car
from world_samples import gen_1, gen_2, generate_random_world_plain
import numpy as np
import math, random
from random import randint
from drivingppo.world import World, Car, OBSTACLE_VALUE, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.environment import SPD_MAX_STD

NEAR = 3.0
W_CONFIG = {
    'near': NEAR,
    'far': 35.0,
}

world = gen_1()

app = WorldController(
    world,
    time_accel=2,
    use_real_time=False,
    frame_delay=33)

In [ ]:
a = 0.996
a**100

In [ ]:
# 모델 1회 실행 결과 확인
from drivingppo.environment import WorldEnv, distance_score_near, distance_score_far

import numpy as np
from stable_baselines3 import PPO

env = WorldEnv(
    world_generator=lambda: World(),
    render_mode='debug',
    auto_close_at_end=False
)
model = PPO.load('./ppo_world_checkpoints/ppo.zip', env=env)

path_data = []

# agent_speed, path_data, obs_near, obs_nearest_angle, obs_nearest_near
np.array([0.0] + path_data, dtype=np.float32)

# ppo 로그 파일(./logs/ppo.csv)에서 obs 부분만 여기 복붙하여 확인
obs = np.array([+8.321,+0.057,+0.091,+0.996,+0.056,+0.082,+0.997,+0.056,+0.073,+0.997,+0.055,+0.064,+0.998,+0.054,+0.055,+0.998,+0.406,+0.714,+0.689,+0.657,+0.616,+0.562,+0.491,+0.394,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000
])

action, _ = model.predict(obs, deterministic=True)

print(action)

In [ ]:
# WorldController 켜기

from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, rad_to_deg
from drivingppo.common import MAP_W, MAP_H, LOOKAHEAD_POINTS
from drivingppo.environment import distance_score_near, distance_score_far, apply_action, observation_str, action_str
from drivingppo.simsim import WorldController

player = Car({
    'playerPos': {'x': 10, 'z': 10},
    "playerSpeed": 0,
    "playerBodyX": 270 * rad_to_deg,
    "playerBodyY": 0,
    "playerBodyZ": 0,
    "playerHealth": 0,
    "playerTurretX": 0,
    "playerTurretY": 0,
})

# obstacle_map, w, h = load_obstacle_map('./map-50.txt')

waypoints = [(10, 100), (100, 150), (150, 100), (200, 200), (250, 250)]
waypoints = []

world = World(
    player=player,
    wh=(MAP_W, MAP_H),
#   obstacle_map=obstacle_map,
    waypoints=waypoints,
    config={
        'angle_start': -pi/4,
        'angle_end': pi/4,
        'use_stop': True
})

WorldController(
    world,
    time_accel=1,
    use_real_time=False,
    frame_delay=33,
    config={
        'TrackingMode': False,
        'LogMode': True,
        'api_delay': 1000
    })

In [ ]:
# 거리점수 확인

import math
import numpy as np
import matplotlib.pyplot as plt
from drivingppo.environment import distance_score_near
from drivingppo.environment import distance_score_far

# def distance_score_near(x:float) -> float:
#     d = x + 10.0
#     x = 100./d/d
#     if x <= 1:
#         return x
#     else:
#         return 1.0

# def distance_score_far(distance:float) -> float:
#     return math.log(distance + 1.0)/10.0

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

xs = np.array(range(3000))/100
nears = np.array([distance_score_near(x) for x in xs])
fars  = np.array([distance_score_far(x)  for x in xs])

print('NEAR')
for x in range(0, 21):
    print(f'{x}: {distance_score_near(x):.4f}')

print('FAR')
for x in range(0, 21):
    print(f'{x}: {distance_score_far(x):.4f}')

axs[0].plot(xs, nears)
axs[0].grid(True)
axs[0].set_yticks([0.0, 0.5, 1.0])
axs[0].set_xticks([0, 1, 2, 10, 20, 30])

axs[1].plot(xs, fars)
axs[1].grid(True)
# axs[1].set_yticks([0.0, 0.5, 1.0])
axs[1].set_xticks([0, 1, 10, 20, 30, 40])

plt.show()